## Card Deck
介绍python中的data model,通过composition的方式充分利用python的核心语言功能,通过python内置的data model,配合它内置的一些方法,比如len(),sorted(),数组切片等等,实现以下功能:
• Collections
• Attribute access
• Iteration (including asynchronous iteration using async for)
• Operator overloading (+-*/)
• Function and method invocation
• String representation and formatting
• Asynchronous programming using await
• Object creation and destruction
• Managed contexts using the with or async with statements

它的优点是方法名统一,并且可以使用内置函数,一些核心功能,它不类似于java中的继承来获取到函数功能,而是通过组合的方式
好处:
1.方法名统一
2.减少重复造轮子

文本中介绍了一个很经典的列子,就是Card Deck列子,通过data model,实现一些Dunder method(double underscored双下划线) 来使其拥有序列的功能,能切割,迭代等等...

In [7]:

from collections import namedtuple

Card = namedtuple('Card', ['rank', 'suit'])  # 使用的namedtuple,第一个是类名,第二个参数是属性名

"""
一般的内置类是不需要手动调用特殊魔法方法, 只有当自定义类,并且需要对元数据操作时,那么需要通过魔法方法来实现功能而不是显式的调用,
而且调用魔法方法时,优先考虑内置方法,因为它提供特殊的额外处理
"""


class Deck:
    # 类成员变量
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()  # 不传分隔符,默认是空格

    # 类对象初始化方法
    def __init__(self):
        self._card = [Card(rank, suit) for rank in self.ranks for suit in  # 两层for循环遍历
                      self.suits]  # 两次迭代生成变量,单下划线表示protected变量,双下划线是私有变量

    # python中的集合list,tuple,dict等集合,它的长度并非通过调用方法获取,而是通过底层c语言编写数据字段b_size来获取,速度快于方法
    def __len__(self):
        return len(self._card)

    # 迭代器调用的方法,首先会找__iter__方法,如果没有,则调用__getitem__方法
    def __getitem__(self, position):
        return self._card[position]


# 因为Deck它实现了__len__()和__getitem__()方法,所以它可以向sequence一样被index,iter,slice,etc
deck = Deck()
deck[0]
deck[12:13]

for card in deck:
    print(card)

# 此方法是隐含的调用的迭代方法,通过__getitem__()方法,来判断是否跟原值相等
flag = Card("Q", "spades") in deck
print(flag)

# 排序
suit_values = {suit: i for i, suit in enumerate(reversed(Deck.suits))}
print(suit_values)


def spades_high(card):
    rank_value = Deck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]


for card in sorted(deck, key=spades_high):
    print(card)



Card(rank='2', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='2', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='clubs')
Card(rank='3', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='clubs')
Card(rank='4', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='clubs')
Card(rank='5', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='clubs')
Card(rank='6', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='clubs')
Card(rank='7', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='clubs')
Card(rank='8', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='clubs')
Card(rank='9', suit='hearts')
Card(rank='10', suit='spades')
C

## Vector
文章再次列举了一个例子,通过实现特殊的方法, __add__,__mul__,__abs__,__bool__,__repr__方法来进一步说明data model,通过Python Data model让自定义类可以实现operator overloading(+-*/)

In [1]:
import math


class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Vector({self.x!r}, {self.y!r})'

    # 实值,能通过内置abs()方法来调用
    def __abs__(self):
        return math.hypot(self.x, self.y)

    # bool context中调用自定义类的此方法
    def __bool__(self):
        return bool(abs(self))

    # +
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y);

    # *
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

## repr和str的区别
__repr__:对于每个自定义的类都必须重写,它是一个二次特性,用来明确说明当前变量所表述的类型值
__str__:用来可读,输出打印字符串时调用,如果自定义没有str方法,会寻找repr方法

如果是输出字符串,调用str,如果是拼接,一定repr方法

变量x
print(x)  : str方法
x : repr方法
print(f"{x}") : str方法
print(f"{x!r}") : repr方法

In [77]:
class Test:
    s = ['1', 2, 3, '4'] # python容器里面的str调用的是元素的repr方法

    # def __repr__(self):
    #     return f"hello world {self.s!r}"  # {self.s!r} = repr{self.s}
    #
    # def __str__(self):
    #     return "str method!"


t = Test()
# t  # 调用的是repr()方法
# print(t) # 调用的是str方法,如果没有重写str,则会使用repr

if t:
    print("hello")

bool()


hello


## bool值
在大部分的bool上下文中,任何对象都可以使用,自定义对象直接返回true,但如果实现了__bool__方法或者__len__方法,则会先调用
__bool__,没有则调用__len__,最后判断是否不等于0,无法转换为数值,直接为True